In [1]:
from get_common_sp500_stock_tickers import get_common_sp500_stock_tickers

In [2]:
from datetime import date
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd

from envs import QUANDL_ACCESS_TOKEN
import quandl
from time import sleep
import random
quandl.ApiConfig.api_key = QUANDL_ACCESS_TOKEN


n_year=5
end_date = date.today()
start_date = end_date - relativedelta(years=n_year)
common_sp500_tickers = get_common_sp500_stock_tickers(start_date,end_date)
common_sp500_tickers = [x.replace(".","_") for x in common_sp500_tickers]

In [4]:
def download_stock(args):
    stock, start_date, end_date = args
    sleep(random.randint(0,1500)/1000)
    try:
#         print(stock)
        output_name = stock + '_data.csv'
        stock_df = quandl.get('WIKI/'+ stock, start_date=start_date.strftime('%Y-%m-%d'), end_date=end_date.strftime('%Y-%m-%d'))
        stock_df.to_csv(output_name)
        return ''
    except Exception as e: 
        bad_names.append(stock)
        print('bad: {}, {}'.format(stock, e))   
        return stock

In [7]:
from datetime import datetime
from multiprocessing import Pool, Array
from itertools import repeat
import tqdm


now_time = datetime.now()
bad_names =[] #to keep track of failed queries

max_workers = 10

workers = min(max_workers, len(common_sp500_tickers)) #in case a smaller number of stocks than threads was passed in
p = Pool(max_workers)
bad_names = list(tqdm.tqdm(p.imap_unordered(download_stock, zip(common_sp500_tickers, repeat(start_date), repeat(end_date))), total=len(common_sp500_tickers)))



""" Save failed queries to a text file to retry """
if len(bad_names) > 0:
    with open('failed_queries.txt','w') as outfile:
        for name in bad_names:
            outfile.write(name+'\n')

#timing:
finish_time = datetime.now()
duration = finish_time - now_time
minutes, seconds = divmod(duration.seconds, 60)
print('getSandP_threaded.py')
print(f'The threaded script took {minutes} minutes and {seconds} seconds to run.')
#The threaded script took 0 minutes and 31 seconds to run.

 17%|█▋        | 67/393 [00:25<01:28,  3.66it/s]

bad: CPRI, (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.


 50%|████▉     | 196/393 [01:05<00:49,  4.02it/s]

bad: JEF, (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.


 55%|█████▍    | 215/393 [01:12<00:55,  3.20it/s]

bad: LIN, (Status 404) (Quandl Error QECx02) You have submitted an incorrect Quandl code. Please check your Quandl codes and try again.


100%|██████████| 393/393 [02:15<00:00,  1.83it/s]

getSandP_threaded.py
The threaded script took 2 minutes and 15 seconds to run.


In [9]:
# pd.core.common.is_list_like = pd.api.types.is_list_like
# import pandas_datareader.data as web
# sp500inx = web.DataReader('^GSPC', 'yahoo', start_date, end_date)
# sp500inx['Date'] = sp500inx.index
sp500inx = pd.read_csv('sp500_index.csv')
sp500inx.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,4/14/14,1818.180054,1834.189941,1815.800049,1830.609985,1830.609985,3111540000
1,4/15/14,1831.449951,1844.020020,1816.290039,1842.979980,1842.979980,3736440000
2,4/16/14,1846.010010,1862.310059,1846.010010,1862.310059,1862.310059,3155080000
3,4/17/14,1861.729980,1869.630005,1856.719971,1864.849976,1864.849976,3341430000
4,4/21/14,1865.790039,1871.890015,1863.180054,1871.890015,1871.890015,2642500000


In [12]:
sp500inx['Date'] = pd.to_datetime(sp500inx['Date'], format='%m/%d/%y')

In [13]:
sp500inx['Date'] = sp500inx['Date'].dt.date
sp500inx = sp500inx[['Date', 'Adj Close']]
sp500inx.columns=['Date', 'SP500']

In [14]:
sp500inx = sp500inx.reset_index(drop=True)
sp500inx.head()

,Date,SP500
0,2014-04-14,1830.609985
1,2014-04-15,1842.979980
2,2014-04-16,1862.310059
3,2014-04-17,1864.849976
4,2014-04-21,1871.890015


In [15]:
import glob
csv_files = glob.glob('individual_stocks/*_data.csv')
file = csv_files[0]
stock_name = file.split('/')[1].split('_')[:-1]
stock_name = '_'.join(stock_name)

In [31]:
import glob
csv_files = glob.glob('individual_stocks/*_data.csv')
file = csv_files[0]
stock_name = file.split('/')[1].split('_')[:-1]
stock_name = '_'.join(stock_name)
df= pd.read_csv(csv_files[0])[['Date', 'Adj. Close']]
df.columns = ['Date', stock_name]
for f in csv_files[1:]:
#     df = pd.merge(df, pd.read_csv(f)[['Date', 'Adj. Close']], how='inner', on='Date')
#     print(df.shape)
    df_ = pd.read_csv(f)[['Date', 'Adj. Close']]
    stock_name = f.split('/')[1].split('_')[:-1]
    stock_name = '_'.join(stock_name)
    df_.columns = ['Date', stock_name]
    if df_.shape[0]>=990:
        df = pd.merge(df, df_, how='left', on='Date')
    else:
        print('skip {} as the shape of the data is ({}, {}), which is less that 995 data point'.format(f.split('_')[0], df_.shape[0], df_.shape[1]))

skip individual as the shape of the data is (481, 2), which is less that 995 data point
skip individual as the shape of the data is (694, 2), which is less that 995 data point
skip individual as the shape of the data is (100, 2), which is less that 995 data point
skip individual as the shape of the data is (141, 2), which is less that 995 data point
skip individual as the shape of the data is (181, 2), which is less that 995 data point
skip individual as the shape of the data is (90, 2), which is less that 995 data point
skip individual as the shape of the data is (231, 2), which is less that 995 data point
skip individual as the shape of the data is (74, 2), which is less that 995 data point
skip individual as the shape of the data is (231, 2), which is less that 995 data point
skip individual as the shape of the data is (4, 2), which is less that 995 data point
skip individual as the shape of the data is (914, 2), which is less that 995 data point
skip individual as the shape of the 

In [35]:
# df.Date[0]
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df['Date'] = df['Date'].dt.date

In [37]:
df = pd.merge(df, sp500inx, how='inner', on='Date')
df = df.set_index('Date') 

In [38]:
print(df.isnull().values.sum())
df

14


,XRX,GS,MTB,V,CTAS,APH,BBT,RHI,HD,PEP,...,HAL,CERN,SJM,WY,WFC,QCOM,COF,COG,F,SP500
Date,,,,,,,,,,,,,,,,,,,,,
2014-04-14,41.508942,147.558327,111.858887,48.989583,53.704614,44.146049,34.920348,37.657543,70.504615,75.634459,...,55.649812,52.870,87.373415,24.427920,43.229405,70.117346,69.191375,32.266936,13.047057,1830.609985
2014-04-15,41.362655,147.729973,111.433110,49.730483,53.733226,44.213802,35.199064,37.770262,70.430370,76.005616,...,56.488139,53.970,87.089467,24.314057,43.831436,70.427443,69.621135,32.473776,13.163400,1842.979980
2014-04-16,42.057519,149.923227,112.590112,51.024621,54.324540,45.191384,35.369889,38.981992,71.070731,76.738876,...,56.987367,53.980,88.692397,24.436679,44.109987,71.038777,70.219062,33.537521,13.354535,1862.310059
2014-04-17,41.801516,150.133017,112.497551,50.680980,54.963540,45.549508,34.102184,39.620733,71.544041,77.444979,...,57.364143,54.000,88.573322,24.410403,43.966219,72.048807,69.966812,34.394427,13.296364,1864.849976
2014-04-21,41.947803,150.476309,112.155079,50.968566,54.734644,45.665656,33.985303,39.658306,72.351452,77.770873,...,59.266862,54.110,88.600801,24.699439,44.136944,71.703271,70.116293,34.591417,13.279743,1871.890015
2014-04-22,42.386664,153.012855,112.877048,51.173289,55.125674,45.980225,34.264018,40.381586,73.938432,77.073823,...,59.700154,53.710,88.783993,24.918405,44.235785,71.419753,70.172349,35.113440,13.379466,1879.550049
2014-04-23,43.739819,153.003319,113.839673,50.893014,55.163823,46.231879,34.210073,40.804283,72.871164,77.318243,...,60.962354,52.890,88.316853,25.049785,44.559264,71.508353,69.994839,35.596065,13.487499,1875.390015
2014-04-24,43.849535,153.384754,113.765625,51.034370,55.383181,46.614201,33.994294,41.414844,74.031238,77.417821,...,59.803768,52.300,88.838950,25.172407,44.316655,68.992137,69.826672,37.398522,13.562291,1878.609985
2014-04-25,42.935241,150.895888,113.145472,48.482652,54.887241,45.835039,33.562734,40.503698,73.669295,77.173401,...,59.210345,49.510,87.950469,25.855583,44.074045,68.761780,68.518707,37.605361,13.113539,1863.400024


In [40]:
pct_df = df.pct_change(fill_method='ffill')
pct_df = pct_df.iloc[1:]
print(pct_df.isnull().values.sum())
pct_df.head()

0


,XRX,GS,MTB,V,CTAS,APH,BBT,RHI,HD,PEP,...,HAL,CERN,SJM,WY,WFC,QCOM,COF,COG,F,SP500
Date,,,,,,,,,,,,,,,,,,,,,
2014-04-15,-0.003524,0.001163,-0.003806,0.015124,0.000533,0.001535,0.007981,0.002993,-0.001053,0.004907,...,0.015064,0.020806,-0.003250,-0.004661,0.013926,0.004423,0.006211,0.006410,0.008917,0.006757
2014-04-16,0.016799,0.014846,0.010383,0.026023,0.011005,0.022110,0.004853,0.032082,0.009092,0.009647,...,0.008838,0.000185,0.018406,0.005043,0.006355,0.008680,0.008588,0.032757,0.014520,0.010488
2014-04-17,-0.006087,0.001399,-0.000822,-0.006735,0.011763,0.007925,-0.035841,0.016386,0.006660,0.009201,...,0.006612,0.000371,-0.001343,-0.001075,-0.003259,0.014218,-0.003592,0.025551,-0.004356,0.001364
2014-04-21,0.003500,0.002287,-0.003044,0.005674,-0.004164,0.002550,-0.003427,0.000948,0.011286,0.004208,...,0.033169,0.002037,0.000310,0.011841,0.003883,-0.004796,0.002136,0.005727,-0.001250,0.003775
2014-04-22,0.010462,0.016857,0.006437,0.004017,0.007144,0.006889,0.008201,0.018238,0.021934,-0.008963,...,0.007311,-0.007392,0.002068,0.008865,0.002239,-0.003954,0.000799,0.015091,0.007509,0.004092


In [41]:
import seaborn as sns
pct_df['Date'] = pct_df.index
sns.lineplot(data=pct_df.melt('Date', var_name='tickers', value_name='vals'), x='Date', y='vals', hue='tickers')

In [42]:
pct_df.to_csv('sp500_pct.csv')

In [43]:
pct_df

,XRX,GS,MTB,V,CTAS,APH,BBT,RHI,HD,PEP,...,CERN,SJM,WY,WFC,QCOM,COF,COG,F,SP500,Date
Date,,,,,,,,,,,,,,,,,,,,,
2014-04-15,-0.003524,0.001163,-0.003806,0.015124,0.000533,0.001535,0.007981,0.002993,-0.001053,0.004907,...,0.020806,-0.003250,-0.004661,0.013926,0.004423,0.006211,0.006410,0.008917,0.006757,2014-04-15
2014-04-16,0.016799,0.014846,0.010383,0.026023,0.011005,0.022110,0.004853,0.032082,0.009092,0.009647,...,0.000185,0.018406,0.005043,0.006355,0.008680,0.008588,0.032757,0.014520,0.010488,2014-04-16
2014-04-17,-0.006087,0.001399,-0.000822,-0.006735,0.011763,0.007925,-0.035841,0.016386,0.006660,0.009201,...,0.000371,-0.001343,-0.001075,-0.003259,0.014218,-0.003592,0.025551,-0.004356,0.001364,2014-04-17
2014-04-21,0.003500,0.002287,-0.003044,0.005674,-0.004164,0.002550,-0.003427,0.000948,0.011286,0.004208,...,0.002037,0.000310,0.011841,0.003883,-0.004796,0.002136,0.005727,-0.001250,0.003775,2014-04-21
2014-04-22,0.010462,0.016857,0.006437,0.004017,0.007144,0.006889,0.008201,0.018238,0.021934,-0.008963,...,-0.007392,0.002068,0.008865,0.002239,-0.003954,0.000799,0.015091,0.007509,0.004092,2014-04-22
2014-04-23,0.031924,-0.000062,0.008528,-0.005477,0.000692,0.005473,-0.001574,0.010468,-0.014435,0.003171,...,-0.015267,-0.005262,0.005272,0.007313,0.001241,-0.002530,0.013745,0.008075,-0.002213,2014-04-23
2014-04-24,0.002508,0.002493,-0.000650,0.002778,0.003976,0.008270,-0.006307,0.014963,0.015920,0.001288,...,-0.011155,0.005912,0.004895,-0.005445,-0.035188,-0.002403,0.050636,0.005545,0.001717,2014-04-24
2014-04-25,-0.020851,-0.016226,-0.005451,-0.050000,-0.008955,-0.016715,-0.012695,-0.022000,-0.004889,-0.003157,...,-0.053346,-0.010001,0.027140,-0.005474,-0.003339,-0.018732,0.005531,-0.033088,-0.008096,2014-04-25
2014-04-28,0.022147,-0.010743,-0.002863,0.012517,0.006082,0.000211,-0.004554,0.011364,0.009952,0.018416,...,-0.001616,0.014997,0.009824,-0.002243,0.005669,-0.005318,0.017287,0.012041,0.003236,2014-04-28
